In [1]:
from typing import Callable
import hashlib

In [2]:
class Super:
    def __init__(self, name: str, H: Callable[[str], str]):
        self.name = name
        self.__H = H
    
    def H(self, s: str, n: int = 1) -> str:
        for _ in range(n):
            s = self.__H(s)
        return s
    
    def display(self, s: str) -> None:
        print(f"{self.name}{s}")

**Загальновідома геш-функція:**

Наприклад, SHA-1.

In [3]:
def hash_f(s: str) -> str:
    return hashlib.sha1(s.encode()).hexdigest()

**Відправник:**

In [4]:
class Sender(Super):
    def __init__(self, name: str, H: Callable[[str], str]):
        super().__init__(name, H)
        self.__w = None
        self.__t = None
        self.__i = None
    
    def register(self, w: str, t: int):
        self.__w = w
        self.__t = t
        self.__i = 1
    
    def send(self, channel: list[(str, int, str)], message: str):
        if self.__w is None:
            self.display(": Помилка надсилання! Не було проведено початкової реєстрації.")
            return
        if self.__i > self.__t - 1:
            self.display(": Помилка надсилання! Кінець послідовності гешування.")
            return
        wi = self.H(self.__w, self.__t - self.__i)
        channel.append((message, self.__i, wi))
        self.display(f" --> channel: (message='{message}', i={self.__i}, wi={wi})")
        self.__i += 1

**Отримувач:**

In [5]:
class Receiver(Super):
    def __init__(self, name: str, H: Callable[[str], str]):
        super().__init__(name, H)
        self.__w0 = None
        self.__ia = None
    
    def register(self, w0: str):
        self.__w0 = w0
        self.__ia = 1
        
    def receive(self, channel: list[(str, int, str)]):
        if self.__w0 is None:
            self.display(": Помилка отримання! Не було проведено початкової реєстрації.")
            return
        message, i, wi = channel.pop(0)
        self.display(f" <-- channel: (message='{message}', i={i}, wi={wi})")
        if self.__ia != i or self.H(wi) != self.__w0:
            self.display(f": Повідомлення '{message}' відхилено.")
            return
        self.display(f": Повідомлення '{message}' прийнято.")
        self.__w0 = wi
        self.__ia += 1

**Третя довірча сторона:**

In [6]:
class Trusted(Super):
    def __init__(self, p: str, t: int, name: str, H: Callable[[str], str]):
        super().__init__(name, H)
        self.__p = p
        self.__t = t
    
    def register(self, sender: Sender, receiver: Receiver):
        sender.register(self.__p, self.__t)
        self.display(f" --> {sender.name}: p='{self.__p}', t={self.__t}")
        w0 = self.H(self.__p, self.__t)
        receiver.register(w0)
        self.display(f" --> {receiver.name}: w0={w0}")

**Відкритий канал:**

In [7]:
open_channel = []

**Опис можливого процесу комунікації:**

In [8]:
t = Trusted("secret password", 10, 'T', hash_f)
a = Sender('A', hash_f)
b = Receiver('B', hash_f)

Етап початкової реєстрації А та B з використанням третьої довірчої сторони Т:

In [9]:
t.register(a, b)

T --> A: p='secret password', t=10
T --> B: w0=f44bbc497bfffa9aaa37a985e0f718e492cea5ba


А надсилає повідомлення "Привіт, B" у відкритий канал:

In [10]:
a.send(open_channel, "Привіт, B")

A --> channel: (message='Привіт, B', i=1, wi=92dac6e4da9348cdfc678e3bdc603d6ec2e8f362)


B читає повідомлення із відкритого каналу та виконує його автентифікацію: 

In [11]:
b.receive(open_channel)

B <-- channel: (message='Привіт, B', i=1, wi=92dac6e4da9348cdfc678e3bdc603d6ec2e8f362)
B: Повідомлення 'Привіт, B' прийнято.


А надсилає друге повідомлення "B, яка у тебе погода?":

In [12]:
a.send(open_channel, "B, яка у тебе погода?")

A --> channel: (message='B, яка у тебе погода?', i=2, wi=10033290799863736bf75a434ef24bdc43251317)


B читає друге повідомлення та виконує його автентифікацію: 

In [13]:
b.receive(open_channel)

B <-- channel: (message='B, яка у тебе погода?', i=2, wi=10033290799863736bf75a434ef24bdc43251317)
B: Повідомлення 'B, яка у тебе погода?' прийнято.


Зловмисник С намагається підробити наступне повідомлення А до B використовуючи індекс та геш з попереднього повідомлення:

In [14]:
message = "У мене от дощ весь день."
i = 3
wi = "10033290799863736bf75a434ef24bdc43251317"
open_channel.append((message, i, wi))
print(f"C --> channel: (message='{message}', i={i}, wi={wi})")

C --> channel: (message='У мене от дощ весь день.', i=3, wi=10033290799863736bf75a434ef24bdc43251317)


B читає третє повідомлення та виконує його автентифікацію:

In [15]:
b.receive(open_channel)

B <-- channel: (message='У мене от дощ весь день.', i=3, wi=10033290799863736bf75a434ef24bdc43251317)
B: Повідомлення 'У мене от дощ весь день.' відхилено.


А надсилає наступне повідомлення до B:

In [16]:
a.send(open_channel, "У мене, наприклад, сонячно.")

A --> channel: (message='У мене, наприклад, сонячно.', i=3, wi=a1a506fe9d030c73173415722a1af85890fdcdda)


B читає повідомлення та виконує його автентифікацію:

In [17]:
b.receive(open_channel)

B <-- channel: (message='У мене, наприклад, сонячно.', i=3, wi=a1a506fe9d030c73173415722a1af85890fdcdda)
B: Повідомлення 'У мене, наприклад, сонячно.' прийнято.
